In [4]:
import sys
sys.path.insert(0, '/Users/ema/query_similarity/adversarial')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from util.text_processor import TextProcessor
from gensim.models.doc2vec import Doc2Vec
import gensim
from sklearn.metrics import accuracy_score

from util.util import get_balanced_data
from word_similarity.glove_synonyms import GloveSynonyms
from word_similarity.word2vec import Word2Vec
from models.word_in_common_feature_extractor import WordsInCommonFeatureExtractor
from models.model import Model
import pickle
# from words_in_common_model import WordsInCommonModel
# from kaggle_model import KaggleModel

%load_ext autoreload

%autoreload 2


pal = sns.color_palette()

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from text_processor import TextProcessor
from gensim.models.doc2vec import Doc2Vec
import gensim
from sklearn.metrics import accuracy_score

from util import get_balanced_data
from word2vec import Word2Vec
from adversarial_algos import adversarial_white_box_change
from word_in_common_feature_extractor import WordsInCommonFeatureExtractor
from model import Model

%load_ext autoreload

%autoreload 2


pal = sns.color_palette()

In [5]:
X_train, X_test, y_train, y_test = get_balanced_data()

We have 298526 positive samples
We have 298526 negative samples


## Feature Extraction

In [6]:
tp = TextProcessor()

In [7]:
feature_extractor = WordsInCommonFeatureExtractor()

## Predictions

In [8]:
clf = LinearRegression()

In [9]:
model = Model(feature_extractor, clf)

In [10]:
%%time
model.fit(X_train, y_train)

(238820, 1)
CPU times: user 53.2 s, sys: 439 ms, total: 53.6 s
Wall time: 54.7 s


/usr/local/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [11]:
y_pred = model.predict(X_test)

In [12]:
log_loss(y_test, y_pred)

0.602438436370617

In [13]:
y_pred_bin = [0 if y < 0.5 else 1 for y in y_pred]

In [14]:
accuracy_score(y_test, y_pred_bin)

0.658124811576726

In [15]:
pickle.dump(model, open('../data/words_in_common/model.dump', 'wb'))

## Load Word2Vec

In [9]:
%%time
glove = GloveSynonyms()

Loading Glove Model
Done. 65713  words loaded!
CPU times: user 9.86 s, sys: 899 ms, total: 10.8 s
Wall time: 13.4 s


In [13]:
glove.most_similar('with', n=10)

['avec', 'among', 's', 'z', 'via', 'from', 'con', 'using', 'into', 'at']

In [8]:
w2v = Word2Vec()

KeyboardInterrupt: 

## Comparison

In [36]:
wic = WordsInCommonModel()

In [37]:
kaggle = KaggleModel()

In [41]:
for idx, row in enumerate(X_test):
    p_a = wic.predict_single(row[0], row[1])
    p_b = kaggle.predict_single(row[0], row[1])
    label = y_test[idx]
    
    if label == 1 and p_a < 0.3 and p_b > 0.5:
        print(row)
        print("wic score: {}".format(p_a))
        print("kaggle score: {}".format(p_b))
        print()
    

['Which is the best cell phone under 10k INR?'
 'Which is the best Android phones under ₹10k?']
wic score: 0.29229880519310025
kaggle score: 0.5206485390663147

['Can you get pregnant one day before ovulation?'
 'Is it possible to get pregnant if a couple is trying just 4 days before her menstrual cycle?']
wic score: 0.299525477672551
kaggle score: 0.8954906463623047

['What is the best laptop under 30k for students?'
 'I want a new laptop with maximum configuration under 30000?']
wic score: 0.2735094567465284
kaggle score: 0.9081634879112244

["Is there any proven method to increase 13 year old girl's height after hitting puberty?"
 'Does height increase after 18 years?']
wic score: 0.28558832360503883
kaggle score: 0.9435322284698486

['How do I get a permanent residence in the US?'
 'How can I get a Legal Permanent Resident status (green card) for the USA?']
wic score: 0.299525477672551
kaggle score: 0.7129401564598083

['Can a girl get pregnant after her last day of periods?'
 'Can

KeyboardInterrupt: 

## Adversarial White Box Algorithm



In [14]:
model = WordsInCommonModel()

In [32]:
t = X_test[20:30]

In [38]:
for row in t:
    print(row)

['Who will win in America presidential elections in 2016?'
 'Who will win the US elections 2016?']
['Can a radiator be used to propel spaceships?'
 'Can heat be used to propel spaceships?']
['How do I change the color of sim tower signal in android?'
 'Why did Android change the background color from black to white? Is a black background not more energy efficient?']
['Which key witnesses supported the death penalty for Dzhokhar Tsarnaev?'
 'Which key witnesses did not support the death penalty for Dzhokhar Tsarnaev?']
['What are the most upsetting and consequential things that you read in the leaked DNC Emails?'
 'What is the best, and most surprising, thing that a friend has done for you?']
['How cold can the Gobi Desert get, and how do its average temperatures compare to the ones in the Great Sandy Desert?'
 'How cold can the Gobi Desert get, and how do its average temperatures compare to the ones in the Simpson Desert?']
['Which are the best places in hauz khas?'
 'What are the best

In [34]:
model.predict(t)

array([0.58756   , 0.66808577, 0.38624555, 0.76203252, 0.16479966,
       0.75480584, 0.70834866, 0.40637699, 0.42147558, 0.66808577])

In [24]:
%%time
results = []
total = 0

for i in range(100):
    q1 = X_test[-i, 0]
    q2 = X_test[-i, 1]
    if model.predict_single(q1, q2) > 0.5:
        total += 1
        print(i)
        results.append(adversarial_white_box_change(q1, q2, model, tp, glove, verbose=1))

0
initial q1: How can I help Syrian refugees?
initial q2: How can everyday people most effectively help the Syrian refugees?
Initial similarity is 0.588

q1: 'How can I help Syrian refugees?'
Updated q2: 'How can everyday people most effectively assistance the Syrian refugees?'
New similarity: 0.421
Replaced word 'help' with 'assistance'
SUCCESSFUL
2
initial q1: Who are some lesser known important historical figures of Jordan and what should people know about them?
initial q2: Who are some lesser known important historical figures of Bolivia and what should people know about them?
Initial similarity is 0.781

q1: 'Who are some lesser known important historical figures of Jordan and what should people know about them?'
Updated q2: 'Who are some smaller known important historical figures of Bolivia and what should people know about them?'
New similarity: 0.642
Replaced word 'lesser' with 'smaller'

q1: 'Who are some lesser known important historical figures of Jordan and what should peop


q1: 'Why is English so necessary, and why is it important in India?'
Updated q2: 'Why English is so momentous in India?'
New similarity: 0.467
Replaced word 'important' with 'momentous'
SUCCESSFUL
33
initial q1: Who was the first president of India?
initial q2: Who was the first president esident of India?
Initial similarity is 0.781

q1: 'Who was the first president of India?'
Updated q2: 'Who was the frst president esident of India?'
New similarity: 0.527
Replaced word 'first' with 'frst'

q1: 'Who was the first president of India?'
Updated q2: 'Who was the frst chairwoman esident of India?'
New similarity: 0.346
Replaced word 'first' with 'frst'
Replaced word 'president' with 'chairwoman'
SUCCESSFUL
35
initial q1: What should I do to control my anger?
initial q2: How can I control my anger? 
Initial similarity is 0.950

q1: 'What should I do to control my anger?'
Updated q2: 'How can I controlling my anger?'
New similarity: 0.527
Replaced word 'control' with 'controlling'

q1: 'Wha


q1: 'What are some unique talent show ideas for children?'
Updated q2: 'What are some singular talent show ideas?'
New similarity: 0.588
Replaced word 'unique' with 'singular'

q1: 'What are some unique talent show ideas for children?'
Updated q2: 'What are some singular talents show ideas?'
New similarity: 0.421
Replaced word 'unique' with 'singular'
Replaced word 'talent' with 'talents'
SUCCESSFUL
60
initial q1: What is a list of the subfields of psychology and what are their distinctions?
initial q2: What are the subfields of psychology?
Initial similarity is 0.612

q1: 'What is a list of the subfields of psychology and what are their distinctions?'
Updated q2: 'What are the subfields of psyche?'
New similarity: 0.386
Replaced word 'psychology' with 'psyche'
SUCCESSFUL
62
initial q1: What is diffusion in biology?
initial q2: How does diffusion work in biology?
Initial similarity is 0.739

q1: 'What is diffusion in biology?'
Updated q2: 'How does spreading work in biology?'
New simi


q1: 'Which is the best real estate agency in pune?'
Updated q2: 'What is the Best actual estate agency in London?'
New similarity: 0.527
Replaced word 'real' with 'actual'

q1: 'Which is the best real estate agency in pune?'
Updated q2: 'What is the Best actual estates agency in London?'
New similarity: 0.386
Replaced word 'real' with 'actual'
Replaced word 'estate' with 'estates'
SUCCESSFUL
CPU times: user 53.7 s, sys: 40.3 s, total: 1min 34s
Wall time: 1min 32s


In [14]:
sum(results) / float(total)

0.9803921568627451